Problem statement.

Build a recommender system by using cosine simillarties score.




In [209]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances

In [210]:
book_df = pd.read_csv('book.csv',encoding='Latin1')
book_df.head()

,Unnamed: 0,User.ID,Book.Title,Book.Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [211]:
book_df.drop(['Unnamed: 0'],axis=1,inplace=True)
book_df.rename({'User.ID':'User_ID','Book.Title':'Book_Title','Book.Rating':'Book_Rating'},axis=1,inplace=True)

In [212]:
book_df[book_df.duplicated()]
book_df.sort_values(['User_ID'])

,User_ID,Book_Title,Book_Rating
2401,8,Wings,5
2400,8,The Western way: A practical guide to the West...,5
2399,8,Ancient Celtic Romances,5
2402,8,Truckers,5
2405,8,The Art Of Celtia,7
...,...,...,...
2395,278854,La crÃ³nica del PerÃº (CrÃ³nicas de AmÃ©rica),7
2398,278854,Celtic Mythology (Library of the World's Myths...,8
2393,278854,A corrente de Trewis Scott,7
2394,278854,As valkÃ­rias,7


In [213]:
print("Total Length of Df : {}".format(len(book_df)))
print("Unique UserId of Book Df : {}".format(len(book_df.User_ID.unique())))
print("Unique Book of Book Df : {}".format(len(book_df.Book_Title.unique())))
print("No. of  Duplicates : {}".format(book_df.drop_duplicates(keep='first', inplace=False).sum()))
book_df = book_df.drop_duplicates(keep='first', inplace=False).reset_index(drop=True)
print("Total Length of Df after dropping duplicates : {}".format(len(book_df)))

Total Length of Df : 10000
Unique UserId of Book Df : 2182
Unique Book of Book Df : 9659
No. of  Duplicates : User_ID                                                953206589
Book_Title     Classical MythologyClara CallanDecision in Nor...
Book_Rating                                                75649
dtype: object
Total Length of Df after dropping duplicates : 9998


In [214]:
#we cannot consider a user who has only registered on the website or has only read one or two books
#So we will limit this number and we will take a user who has rated at least 50 books
book_df['User_ID'].value_counts()
#Extract users and ratings of more than 50
x = book_df['User_ID'].value_counts() > 50
y = x[x].index  #user_ids : getting userId value
print(y.shape)
book_df = book_df[book_df['User_ID'].isin(y)]#lsit of all book details with user who have given more than 50 ratings 
book_df

(27,)


,User_ID,Book_Title,Book_Rating
520,277203,The Fly on the Wall,8
521,277203,Cranford (The World's Classics),8
522,277203,The Red Tent (Bestselling Backlist),8
523,277203,The Right Hand of Amon,8
524,277203,Last Blue: Poems,8
...,...,...,...
9962,162052,Ali and Nino: A Love Story,5
9963,162052,Objects in Mirror Are Closer Than They Appear ...,7
9964,162052,Ragtime,9
9965,162052,The Man Who Ate the 747,10


In [215]:
book_df.Book_Rating.unique()

array([ 8,  9,  6,  7, 10,  5,  4,  2,  3,  1], dtype=int64)

In [216]:
user_book_df =  book_df.pivot_table(index='User_ID',columns='Book_Title',values='Book_Rating').reset_index(drop=True)
user_book_df
# converting long data into wide data using pivot table

Book_Title,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,"1,401 More Things That P*Ss Me Off",1001 Brilliant Ways to Checkmate,101 Essential Tips: House Plants,101 Wacky Computer Jokes,11 Edward Street,2010: Odyssey Two,22 Indigo Place,253,28 Barbary Lane : A \Tales of the City\ Omnibus,...,ZANZIBAR CAT,Zen Meditations on Being a Mother (Zen Meditations),Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zen: A Way of Life (Teach Yourself Books),Zodiac: The Eco-Thriller,Zombie!,"\More More More,\ Said the Baby",\O\ Is for Outlaw,"\Well, there's your problem\: Cartoons",Ã?Â?bermorgen.
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN


In [217]:
user_book_df.index = book_df.User_ID.unique()
user_book_df.fillna(0, inplace=True)
user_book_df

Book_Title,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,"1,401 More Things That P*Ss Me Off",1001 Brilliant Ways to Checkmate,101 Essential Tips: House Plants,101 Wacky Computer Jokes,11 Edward Street,2010: Odyssey Two,22 Indigo Place,253,28 Barbary Lane : A \Tales of the City\ Omnibus,...,ZANZIBAR CAT,Zen Meditations on Being a Mother (Zen Meditations),Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zen: A Way of Life (Teach Yourself Books),Zodiac: The Eco-Thriller,Zombie!,"\More More More,\ Said the Baby",\O\ Is for Outlaw,"\Well, there's your problem\: Cartoons",Ã?Â?bermorgen.
277203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


In [218]:
user_book_df.isna().sum()

Book_Title
...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR    0
1,401 More Things That P*Ss Me Off                              0
1001 Brilliant Ways to Checkmate                                0
101 Essential Tips: House Plants                                0
101 Wacky Computer Jokes                                        0
                                                               ..
Zombie!                                                         0
\More More More,\ Said the Baby                                 0
\O\ Is for Outlaw                                               0
\Well, there's your problem\: Cartoons                          0
Ã?Â?bermorgen.                                                  0
Length: 2888, dtype: int64

In [238]:
user_book1 = 1 - pairwise_distances( user_book_df.values,metric='cosine')

In [239]:
user_book2 = pd.DataFrame(user_book1)
user_book2.index = book_df.User_ID.unique()
user_book2.columns = book_df.User_ID.unique()

In [240]:
user_book2.head()

,277203,277427,278137,278188,278418,183,254,638,643,1211,...,2766,2891,2977,3363,3757,3827,4017,161041,161936,162052
277203,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
277427,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278137,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016945
278188,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278418,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [241]:
np.fill_diagonal(user_book1,0)
user_book2.idxmax(axis=1)[0:5]#idxmax() function returns index of first occurrence of maximum over requested axis..idxmax(axis=1)[0:5]#idxmax() function returns index of first occurrence of maximum over requested axis.

277203    277203
277427    277203
278137    162052
278188    277203
278418    277203
dtype: int64

In [249]:
# extract the books which userId 277203 & 162052 have watched
book_df[(book_df['User_ID']==277203) | (book_df['User_ID']==162052)]

,User_ID,Book_Title,Book_Rating
520,277203,The Fly on the Wall,8
521,277203,Cranford (The World's Classics),8
522,277203,The Red Tent (Bestselling Backlist),8
523,277203,The Right Hand of Amon,8
524,277203,Last Blue: Poems,8
...,...,...,...
9962,162052,Ali and Nino: A Love Story,5
9963,162052,Objects in Mirror Are Closer Than They Appear ...,7
9964,162052,Ragtime,9
9965,162052,The Man Who Ate the 747,10


In [256]:
user_1 = book_df[(book_df['User_ID']==277203)]
user_2 = book_df[(book_df['User_ID']==162052)]
pd.merge(user_1,user_2,on='Book_Title',how='outer')

,User_ID_x,Book_Title,Book_Rating_x,User_ID_y,Book_Rating_y
0,277203.0,The Fly on the Wall,8.0,NaN,NaN
1,277203.0,Cranford (The World's Classics),8.0,NaN,NaN
2,277203.0,The Red Tent (Bestselling Backlist),8.0,NaN,NaN
3,277203.0,The Right Hand of Amon,8.0,NaN,NaN
4,277203.0,Last Blue: Poems,8.0,NaN,NaN
...,...,...,...,...,...
261,NaN,Ali and Nino: A Love Story,NaN,162052.0,5.0
262,NaN,Objects in Mirror Are Closer Than They Appear ...,NaN,162052.0,7.0
263,NaN,Ragtime,NaN,162052.0,9.0
264,NaN,The Man Who Ate the 747,NaN,162052.0,10.0


In [254]:
pd.merge(user_1,user_2,on='Book_Title',how='inner')

,User_ID_x,Book_Title,Book_Rating_x,User_ID_y,Book_Rating_y
